In [2]:
import math
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D
import yfinance as yf

2024-08-24 18:36:26.333345: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-24 18:36:26.400340: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 18:36:27.342323: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 18:36:27.771137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 18:36:28.407456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [3]:
def calculate_bollinger_bands(data, window = 10, num_of_std = 2):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return upper_band, lower_band

def calculate_rsi(data, window=10):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_roc(data, periods=10):
    """Calculate Rate of Change"""
    roc = ((data - data.shift(periods)) / data.shift(periods)) * 100
    return roc

In [7]:
tickers = ['META', 'AAPL', 'MSFT', 'AMZN', 'GOOG']

ticker_data_frames = []
stats = {}

for ticker in tickers:
    data = yf.download(ticker, period='6mo', interval='1d')

    close = data['Close']
    upper, lower = calculate_bollinger_bands(close, window=14, num_of_std=2)
    width = upper - lower
    rsi = calculate_rsi(close, window=14)
    roc = calculate_roc(close, periods=14)
    volume = data['Volume']
    diff = data['Close'].diff(1)
    percent_change_close = data['Close'].pct_change() * 100

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
ticker_df = pd.DataFrame({
    ticker + '_close': close,
    ticker + '_width': width,
    ticker + '_rsi': rsi,
    ticker + '_roc': roc,
    ticker + '_volume': volume,
    ticker + '_diff': diff,
    ticker + '_percent_change_close': percent_change_close
})

MEAN = ticker_df.mean()
STD = ticker_df.std()

for column in MEAN.index:
    stats[f"{column}_mean"] = MEAN[column]
    stats[f"{column}_std"] = STD[column]

ticker_df = (ticker_df - MEAN) / STD
ticker_data_frames.append(ticker_df)

In [15]:
df = pd.concat(ticker_data_frames, axis = 1)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.head()

,GOOG_close,GOOG_width,GOOG_rsi,GOOG_roc,GOOG_volume,GOOG_diff,GOOG_percent_change_close,GOOG_close,GOOG_width,GOOG_rsi,GOOG_roc,GOOG_volume,GOOG_diff,GOOG_percent_change_close
Date,,,,,,,,,,,,,,
2024-03-15,-1.616925,-0.360453,-0.015360,-0.000123,2.428744,-0.836978,-0.957118,-1.616925,-0.360453,-0.015360,-0.000123,2.428744,-0.836978,-0.957118
2024-03-18,-1.194972,0.246472,0.378017,0.553991,3.198996,2.121034,2.450832,-1.194972,0.246472,0.378017,0.553991,3.198996,2.121034,2.450832
2024-03-19,-1.232419,0.631664,0.635045,0.814229,-0.267105,-0.275373,-0.311151,-1.232419,0.631664,0.635045,0.814229,-0.267105,-0.275373,-0.311151
2024-03-20,-1.114728,1.067091,0.596888,0.727497,-0.269236,0.533892,0.587609,-1.114728,1.067091,0.596888,0.727497,-0.269236,0.533892,0.587609
2024-03-21,-1.177585,1.315147,0.699500,0.827967,-0.024418,-0.407922,-0.455026,-1.177585,1.315147,0.699500,0.827967,-0.024418,-0.407922,-0.455026


In [16]:
labels = df.shift(-1)

df = df.iloc[:-1]
labels = labels.iloc[:-1]

In [ ]:
SEQUENCE_LEN = 24